In [1]:
# Used to track the request processing time
import time

from textwrap import dedent
from langchain_community.llms import Ollama
from crewai import Crew, Agent, Task

In [2]:
# Initialize Llama3 LLM for agents.
# This will access local models in the standard installation directory

llm = Ollama(model="NEW_MODEL")

In [3]:
# Uncomment the below line to test if the loaded model
# works and is indeed a custom model.

# llm.invoke("Tell me a joke")

# Create Agents

In [4]:
class FunctionTestAgents:

    def variable_name_agent(self):
        return Agent(
            role='Variable Name Agent',
            goal='Ensure that variable names are descriptive, concise, and accurate.',
            backstory=dedent(
                '''
                You're an expect agent that helps developers write clean, efficient code.
                Your job is to ensure that variable names are descriptive, concise, and accurate.
                Do not add comments to the code.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )

    def code_comment_agent(self):
        return Agent(
            role='Code Comment Agent',
            goal='Add comments to code to explain its purpose and functionality. Only add comments that are necessary for a user to understand the code.',
            backstory=dedent(
                '''
                You're an expect code commentor. Your job is to add comments to the code to explain its purpose and functionality. Only add comments that
                are necessary for a user to understand the code. You're aware that code should be self-explanatory whenever possible, and that comments
                should only be used to explain complex or non-obvious parts of the code.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )

    def documentation_agent(self):
        return Agent(
            role='Documentation Agent',
            goal='Document code in a way that is easy to understand and follow.',
            backstory=dedent(
                '''
                You're an expert in writing docstrings. Your documentation is clear, concise, accurate, and easy to understand.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )

# Create Tasks

In [5]:
class FunctionTestTasks:

    def improve_variable_names(self, agent, request):
        return Task(
            description=dedent(
                f'''
                Receive Python code. Ensure that all variable names are descriptive, concise, and accurate.
                Do not add comments to the code.

                Code
                ----
                {request}
                '''),
            expected_output=dedent(
                '''
                Python code with improved variable names
                '''),
                agent=agent
        )

    def add_code_comments(self, agent, request):
        return Task(
            description=dedent(
                f'''
                Receive Python code. Add comments to the code to explain its purpose and functionality.
                Only add comments that are necessary for a user to understand the code. If a line of
                code is self-explanatory, do not add a comment.

                Code
                ----
                {request}
                '''),
            expected_output=dedent(
                '''
                Python code with added comments
                '''),
                agent=agent
        )

    def review_code_comments(self, agent, request):
        return Task(
            description=dedent(
                f'''
                Receive Python code with comments. Review the code and ensure that all comments are
                accurate and helpful. If a comment is not necessary, remove it.

                Code
                ----
                {request}
                '''),
            expected_output=dedent(
                '''
                Python code with reviewed comments
                '''),
                agent=agent
        )

    def add_documentation(self, agent, request):
        return Task(
            description=dedent(
                f'''
                Add docstrings to the code to explain its purpose and functionality.
                Ensure the documentation is clear, concise, accurate, and easy to understand.

                Code
                ----
                {request}
                '''),
            expected_output=dedent(
                '''
                Python code with added documentation
                '''),
                agent=agent
        )

# Processing Functions
Start reading a file of functions, each preceeded by a comment explaining what the function is; extract the function and feed it through the LLM; then write the result to a new file.

In [6]:
# Define relative file path
test_suite_file_path = 'test_suite/test_functions_raw.py'

# Save all captured functions to a list
functions = []

with open(test_suite_file_path, 'r', encoding='utf-8') as file:

    function_code      = ''
    capturing_function = False
    
    for line in file:
        
        # Check if the line is blank
        if line.strip() == '':
            
            # If the line is blank and we're capturing
            # a function, we've reached the end of the
            # current function; reset
            if capturing_function:
                functions.append(function_code.strip())
                function_code = ''
                capturing_function = False
        
        # If line isn't blank, start capturing function
        else:
            function_code += line
            capturing_function = True
    
    # If the file doesn't end in a blank line,
    # ensure the last function is saved.
    if capturing_function:
        functions.append(function_code.strip())

# Main

In [7]:
# Initialize Agents, Tasks
agents = FunctionTestAgents()
tasks = FunctionTestTasks()


# CREATE AGENTS
variable_name_agent = agents.variable_name_agent()
# code_comment_agent = agents.code_comment_agent()
documentation_agent = agents.documentation_agent()

print("##################################")
print("## Welcome to the Function Crew ##")
print("##################################")

# CREATE TASKS

updated_functions = []

for func in functions:

    improve_variable_names_task = tasks.improve_variable_names(variable_name_agent, func)
    # add_code_comments_task = tasks.add_code_comments(variable_name_agent, improve_variable_names_task)
    add_documentation_task = tasks.add_documentation(variable_name_agent, improve_variable_names_task)


    # CREATE CREW & ASSIGN TASKS
    crew = Crew(
        agents = [
            variable_name_agent,
            documentation_agent
        ],
        tasks = [
            improve_variable_names_task,
            add_documentation_task
        ],
        verbose=True
    )

    # Start code-improvement process
    start_time = time.perf_counter()
    output = crew.kickoff()
    end_time = time.perf_counter()
    
    updated_functions.append(output.strip())

    print("################################")
    print("## Refactored Function Output ##")
    print("################################")
    print(f">>> Time taken: {end_time - start_time:.2f} seconds <<<\n")
    
    
# WRITE FUNCTIONS TO FILE FOR REVIEW

# Use to write a file with the current datetime
import datetime
import os
    
# Check if directory exists; if not, create it
# This directory is relative to the location of this Jupyter Notebook
save_location_file_path = 'test_outputs'
if not os.path.exists(save_location_file_path):
    os.makedirs(save_location_file_path)

# Create file to write output

# Create custom datetime output, used for filename
current_datetime = datetime.datetime.now().strftime('%y-%m-%d_%H:%M:%S') + '.txt' 
output_filename  = 'function_output_' + current_datetime
full_file_path   = os.path.join(save_location_file_path, output_filename)

with open(full_file_path, 'w', encoding='utf-8') as f:
    for func in updated_functions:
        
        # Remove backticks and any remaining whitespaces from function
        cleaned_func = func.strip('`').strip()
        f.write(cleaned_func + "\n\n")
            
print(f'Functions successfully written to file "{full_file_path}".')

##################################
## Welcome to the Function Crew ##
##################################
 [DEBUG]: == Working Agent: Variable Name Agent
 [INFO]: == Starting Task: 
                Receive Python code. Ensure that all variable names are descriptive, concise, and accurate.
                Do not add comments to the code.

                Code
                ----
                # Calculate Fibonacci sequence
def f(n):
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a + b
    return a



> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer:
```
# Calculate Fibonacci sequence
def fibonacci_sequence(n):
    first_number, second_number = 0, 1
    for _ in range(n):
        first_number, second_number = second_number, first_number + second_number
    return first_number
```

> Finished chain.
 [DEBUG]: == [Variable Name Agent] Task output: ```
# Calculate Fibonacci sequence
def fibonacci_sequence(n):
    first_number, second

2024-05-15 16:06:58,038 - 8450341568 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


I now can give a great answer!

Final Answer:
```
# Calculate Fibonacci sequence
def fibonacci_sequence(n):
    """
    Generate the Fibonacci sequence up to the nth number.

    Args:
        n (int): The number of terms in the sequence.

    Returns:
        int: The nth term in the Fibonacci sequence.
    """
    first_number, second_number = 0, 1
    for _ in range(n):
        first_number, second_number = second_number, first_number + second_number
    return first_number
```

> Finished chain.
 [DEBUG]: == [Variable Name Agent] Task output: ```
# Calculate Fibonacci sequence
def fibonacci_sequence(n):
    """
    Generate the Fibonacci sequence up to the nth number.

    Args:
        n (int): The number of terms in the sequence.

    Returns:
        int: The nth term in the Fibonacci sequence.
    """
    first_number, second_number = 0, 1
    for _ in range(n):
        first_number, second_number = second_number, first_number + second_number
    return first_number
```


#####

2024-05-15 16:07:34,546 - 8450341568 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


Thought: I now can give a great answer.

Final Answer:
```
# Merge two sorted lists
def merge_sorted_lists(left, right):
    """
    This function merges two sorted lists into one sorted list.
    
    Parameters:
        left (list): The first sorted list.
        right (list): The second sorted list.
    
    Returns:
        result (list): The merged sorted list.
    """
    result  = []
    left_index, right_index  = 0, 0
    while left_index  < len(left) and right_index  < len(right):
        if left[left_index]  < right[right_index]:
            result.append(left[left_index])
            left_index  += 1
        else:
            result.append(right[right_index])
            right_index  += 1
    result.extend(left[left_index:])
    result.extend(right[right_index:])
    return result
```

> Finished chain.
 [DEBUG]: == [Variable Name Agent] Task output: ```
# Merge two sorted lists
def merge_sorted_lists(left, right):
    """
    This function merges two sorted lists into one s

...

2024-05-15 16:12:42,274 - 8450341568 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


I now can give a great answer!

Final Answer:
```
def retrieve_data(url):
    """
    Retrieves data from a given URL.

    Args:
        url (str): The URL to retrieve data from.

    Returns:
        dict or None: The retrieved data in JSON format, or an empty dictionary if the request fails.
    """
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}
```

> Finished chain.
 [DEBUG]: == [Variable Name Agent] Task output: ```
def retrieve_data(url):
    """
    Retrieves data from a given URL.

    Args:
        url (str): The URL to retrieve data from.

    Returns:
        dict or None: The retrieved data in JSON format, or an empty dictionary if the request fails.
    """
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}
```


################################
## Refactored Function Output ##
################################
>>> Ti

2024-05-15 16:13:50,826 - 8450341568 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


I now can give a great answer!

Final Answer:
```python
import os
import re

"""
This module performs a file search with regex.
It searches for files containing a specific keyword in the specified directory and its subdirectories.

Functions:
    - file_search(): Searches for files containing the keyword and returns a list of matching files.
"""

class VariableNameAgent:
    def __init__(self):
        self.used_tools = 0
        self.tools_errors = 0
        self.delegations = 0
        self.i18n = I18N(prompt_file=None)
        self.thread = None
        self.prompt_context = None
        self.description = '\nReceive Python code. Ensure that all variable names are descriptive, concise, and accurate.\nDo not add comments to the code.\n\nCode\n----\n# Performing a file search with regex\nimport os\nimport re\n' expected_output='\nPython code with improved variable names\n' config=None callback=None agent=Agent(role=Variable Name Agent, goal=Ensure that variable names are descriptive, 

Thought: I'm ready to provide a great answer!

Final Answer:
```
def process_path(path: str, regex: str) -> list:
    """
    Process a file path and return a list of matching files.

    Args:
        path (str): The file path to process.
        regex (str): The regular expression pattern to match.

    Returns:
        list: A list of matching files.
    """
    matches = []  # Initialize an empty list to store the matches
    pattern = re.compile(regex)  # Compile the regular expression pattern

    for directory, _, files in os.walk(path):  # Iterate over the file path
        for file in files:
            if pattern.search(file):  # Check if the file matches the regex pattern
                matches.append(os.path.join(directory, file))  # Add the matching file to the list
    return matches  # Return the list of matching files
```

> Finished chain.
 [DEBUG]: == [Variable Name Agent] Task output: ```
def process_path(path: str, regex: str) -> list:
    """
    Process a file pa